#                                            Import Liberaries

In [1]:
!pip install pandas langdetect
!pip install regex
!pip install obsei dateparser
!pip install pandas numpy tensorflow


In [2]:
import pandas as pd
import re
import numpy as np
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('stopwords')
from langdetect import detect
import emoji
import warnings
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
from sklearn.preprocessing import LabelEncoder
nltk.download('omw-1.4')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashua\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ashua\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
data=pd.read_csv("Youtubecomment.csv")
data.head()

,Channel URL,Comment,Category
0,http://www.youtube.com/channel/UCPox5oO3ubH_TQ...,I think this is the last straw that break the ...,NaN
1,http://www.youtube.com/channel/UCxhvzuxbwK0Ncn...,The US needs to learn from China to become a w...,NaN
2,http://www.youtube.com/channel/UCf5kNNhD3jE8F3...,Thanks for the old news,NaN
3,http://www.youtube.com/channel/UCECVGJCqYazz4d...,Joe Biden and his administration Finance this ...,NaN
4,http://www.youtube.com/channel/UCSGosVOwjsSu5N...,"news sung by Vanshika Jaral, with beautiful pr...",NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45172 entries, 0 to 45171
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Channel URL  28534 non-null  object 
 1   Comment      28534 non-null  object 
 2   Category     0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 1.0+ MB


# preprocessing of YouTube Comments

In [5]:
# Function to remove non-English text
def remove_non_english(text):
    try:
        if detect(text) == 'en':
            return text
        else:
            return ''
    except:
        return ''

# Function to remove punctuation
def remove_punctuation(text):
    # Removes punctuation and replaces it with a space
    return re.sub(r'[^\w\s]', ' ', text)
# Apply the cleaning function to the 'Text' column
data['Comment'] = data['Comment'].apply(remove_non_english)
data['Comment'] = data['Comment'].apply(remove_punctuation)

# Remove rows with empty or whitespace-only text
data = data[data['Comment'].str.strip() != '']
data.to_csv('clean_data.csv',index=True)

In [6]:
dataclean=pd.read_csv('clean_data.csv')
dataclean

,Unnamed: 0,Channel URL,Comment,Category
0,0,http://www.youtube.com/channel/UCPox5oO3ubH_TQ...,I think this is the last straw that break the ...,NaN
1,1,http://www.youtube.com/channel/UCxhvzuxbwK0Ncn...,The US needs to learn from China to become a w...,NaN
2,2,http://www.youtube.com/channel/UCf5kNNhD3jE8F3...,Thanks for the old news,NaN
3,3,http://www.youtube.com/channel/UCECVGJCqYazz4d...,Joe Biden and his administration Finance this ...,NaN
4,4,http://www.youtube.com/channel/UCSGosVOwjsSu5N...,news sung by Vanshika Jaral with beautiful pr...,NaN
...,...,...,...,...
21894,45066,http://www.youtube.com/channel/UCnT3-g1tO2r9V_...,Coward army,NaN
21895,45069,http://www.youtube.com/channel/UCDB0yM3ffYdFj1...,Innocent people should never be punished for r...,NaN
21896,45161,http://www.youtube.com/channel/UCsYMcOEmOyhcSF...,We condemn Israel 39 s fascist racist bigot...,NaN
21897,45163,http://www.youtube.com/channel/UC_pfrX7p3c_f2S...,It 39 s so heart touching scene i am overwhel...,NaN


# Data Labelling

In [7]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
dataclean["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in dataclean["Comment"]]
dataclean["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in dataclean["Comment"]]
dataclean["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in dataclean["Comment"]]
dataclean['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in dataclean["Comment"]]
score = dataclean["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
dataclean["Sentiment"] = sentiment
dataclean.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ashua\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Unnamed: 0,Channel URL,Comment,Category,Positive,Negative,Neutral,Compound,Sentiment
0,0,http://www.youtube.com/channel/UCPox5oO3ubH_TQ...,I think this is the last straw that break the ...,NaN,0.000,0.000,1.000,0.0000,Neutral
1,1,http://www.youtube.com/channel/UCxhvzuxbwK0Ncn...,The US needs to learn from China to become a w...,NaN,0.150,0.208,0.641,-0.4885,Negative
2,2,http://www.youtube.com/channel/UCf5kNNhD3jE8F3...,Thanks for the old news,NaN,0.420,0.000,0.580,0.4404,Positive
3,3,http://www.youtube.com/channel/UCECVGJCqYazz4d...,Joe Biden and his administration Finance this ...,NaN,0.061,0.236,0.703,-0.6920,Negative
4,4,http://www.youtube.com/channel/UCSGosVOwjsSu5N...,news sung by Vanshika Jaral with beautiful pr...,NaN,0.245,0.000,0.755,0.5994,Positive


In [8]:
datafinal=dataclean.drop(['Positive','Negative','Neutral','Compound'],axis=1)
datafinal.head()

,Unnamed: 0,Channel URL,Comment,Category,Sentiment
0,0,http://www.youtube.com/channel/UCPox5oO3ubH_TQ...,I think this is the last straw that break the ...,NaN,Neutral
1,1,http://www.youtube.com/channel/UCxhvzuxbwK0Ncn...,The US needs to learn from China to become a w...,NaN,Negative
2,2,http://www.youtube.com/channel/UCf5kNNhD3jE8F3...,Thanks for the old news,NaN,Positive
3,3,http://www.youtube.com/channel/UCECVGJCqYazz4d...,Joe Biden and his administration Finance this ...,NaN,Negative
4,4,http://www.youtube.com/channel/UCSGosVOwjsSu5N...,news sung by Vanshika Jaral with beautiful pr...,NaN,Positive


In [9]:
datafinal1=datafinal.drop(['Unnamed: 0','Category'],axis=1)
datafinal1.to_csv('final_data.csv',index=True)


In [10]:
datafinal1.head()

,Channel URL,Comment,Sentiment
0,http://www.youtube.com/channel/UCPox5oO3ubH_TQ...,I think this is the last straw that break the ...,Neutral
1,http://www.youtube.com/channel/UCxhvzuxbwK0Ncn...,The US needs to learn from China to become a w...,Negative
2,http://www.youtube.com/channel/UCf5kNNhD3jE8F3...,Thanks for the old news,Positive
3,http://www.youtube.com/channel/UCECVGJCqYazz4d...,Joe Biden and his administration Finance this ...,Negative
4,http://www.youtube.com/channel/UCSGosVOwjsSu5N...,news sung by Vanshika Jaral with beautiful pr...,Positive


In [11]:
datafinal1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21899 entries, 0 to 21898
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Channel URL  21899 non-null  object
 1   Comment      21899 non-null  object
 2   Sentiment    21899 non-null  object
dtypes: object(3)
memory usage: 513.4+ KB


# Data transformation

In [12]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [13]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

def remove_repeated_words(text):
    # Use regular expressions to find and remove repeated words
    return re.sub(r'\b(\w+)\s+\1\b', r'\1', text)
datafinal1['Comment'] = datafinal1['Comment'].apply(remove_repeated_words)

In [15]:
nltk.download('wordnet')
data_copy = datafinal1.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_processing(text))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ashua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [17]:
processed_data = {
    'Sentence':data_copy.Comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,think last straw break camel back hamas,1
1,u need learn china become war free global econ...,0
2,thanks old news,2
3,joe biden administration finance war israel 39...,0
4,news sung vanshika jaral beautiful professiona...,2


In [18]:
processed_data['Sentiment'].value_counts()

Sentiment
0    11033
2     6832
1     4034
Name: count, dtype: int64

In [20]:

df_neutral = processed_data[(processed_data['Sentiment']==1)] 
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_positive_upsampled = resample(df_positive, 
                                 replace=True,    
                                 n_samples= 10000, 
                                 random_state=42)  

df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,    
                                 n_samples= 10000, 
                                 random_state=42)  


# Concatenate the upsampled dataframes with the neutral dataframe
final_data2 = pd.concat([df_positive_upsampled,df_neutral_upsampled,df_negative])

In [89]:
final_data2['Sentiment'].value_counts()

Sentiment
0    11033
2    10000
1    10000
Name: count, dtype: int64

In [90]:
corpus = []
for sentence in final_data2 ['Sentence']:
    corpus.append(sentence)
corpus[0:5]

['pray everyday isreal palestine innocent civilian deserve treated like men keeping war going never frontlines',
 'united state stand palestine people',
 'thank cnn report pray israel',
 'may allah swt protect brother sister palestine ameen',
 'quot music festival quot holy land holy people quot exalt god quot']

In [91]:
final_data2.Sentiment.unique()

array([2, 1, 0])

In [92]:
TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(final_data2['Sentence'])
y = final_data2['Sentiment']

In [93]:
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [94]:
print("training accuracy :",model.score(X_train,y_train))

training accuracy : 0.8259486022718118


In [95]:
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.78


In [96]:
print(classification_report(y_test, y_pred, target_names=['neutral', 'negative', 'positive']))

              precision    recall  f1-score   support

     neutral       0.70      0.86      0.77      2162
    negative       0.84      0.73      0.79      2053
    positive       0.82      0.73      0.78      1992

    accuracy                           0.78      6207
   macro avg       0.79      0.77      0.78      6207
weighted avg       0.79      0.78      0.78      6207



In [97]:
from sklearn.linear_model import LogisticRegression
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(final_data2['Sentence'])
y = final_data2['Sentiment']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
from sklearn.model_selection import cross_val_score
k = 5  # Set the number of folds
scores = cross_val_score(model, X_train, y_train, cv=k, scoring='accuracy')
# Print the cross-validation results
for fold, score in enumerate(scores, start=1):
    print(f'Fold {fold} - Accuracy: {score:.2f  }')
mean_accuracy = scores.mean()
std_deviation = scores.std()
print(f'Mean Accuracy: {mean_accuracy:.2f}')
print(f'Standard Deviation: {std_deviation:.2f}')

Fold 1 - Accuracy: 0.86
Fold 2 - Accuracy: 0.85
Fold 3 - Accuracy: 0.85
Fold 4 - Accuracy: 0.86
Fold 5 - Accuracy: 0.85
Mean Accuracy: 0.85
Standard Deviation: 0.01


In [124]:
y_pred = model.predict(X_test)
final_data2['Predicted Sentiment'] = model.predict(X)
final_data2

,Sentence,Sentiment,Predicted Sentiment
2388,pray everyday isreal palestine innocent civili...,2,2
18442,united state stand palestine people,2,2
17924,thank cnn report pray israel,2,2
17816,may allah swt protect brother sister palestine...,2,2
13124,quot music festival quot holy land holy people...,2,2
...,...,...,...
21884,israel action br palestine suicidal,0,1
21890,haitian living horror 3 year intense terrorist...,0,0
21894,coward army,0,1
21896,condemn israel 39 fascist racist bigotry barba...,0,0


In [100]:
print(classification_report(y_test, y_pred, target_names=['neutral', 'negative', 'positive']))

              precision    recall  f1-score   support

     neutral       0.88      0.83      0.85      2162
    negative       0.85      0.93      0.89      2053
    positive       0.88      0.85      0.87      1992

    accuracy                           0.87      6207
   macro avg       0.87      0.87      0.87      6207
weighted avg       0.87      0.87      0.87      6207



In [101]:
# using random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [109]:
# Perform K-fold cross-validation
from sklearn.model_selection import cross_val_score
k = 5  # Set the number of folds
scores = cross_val_score(random_forest, X_train, y_train, cv=k, scoring='accuracy')
# Print the cross-validation results
for fold, score in enumerate(scores, start=1):
    print(f'Fold {fold} - Accuracy: {score:.2f}')

Fold 1 - Accuracy: 0.90
Fold 2 - Accuracy: 0.90
Fold 3 - Accuracy: 0.90
Fold 4 - Accuracy: 0.90
Fold 5 - Accuracy: 0.89


In [117]:
y_pred_rand = random_forest.predict(X_test)
accuracy = accuracy_score(y_test,y_pred_rand)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.92


In [118]:
print(classification_report(y_test, y_pred_rand, target_names=['neutral', 'negative', 'positive']))

              precision    recall  f1-score   support

     neutral       0.91      0.87      0.89      2162
    negative       0.91      0.97      0.94      2053
    positive       0.93      0.90      0.92      1992

    accuracy                           0.92      6207
   macro avg       0.92      0.92      0.92      6207
weighted avg       0.92      0.92      0.91      6207

